# Lecture 10-3

# Messing around with LLMs with Haystack

Step 0: Get OpenAI API Key

To get an OpenAI API key, you need to sign up for an OpenAI account and subscribe to their API service. 

- Go to the OpenAI website.
- Click on "Sign Up" to create a new account, or "Log In" if you already have an account.
- Click on "API" at the top of the page.
- You may need to set up a billing plan. If you stick with gpt 3.5, it is cheap. You can buy $5 of credits, which will be more than enough for experimentation.
- Go to the API keys section of your OpenAI account.
- Click on "Create new secret key" to generate a new API key.
- Copy the generated API key.
- Create a `.env` file in your project directory.
- In the `.env` file, enter:

```
OPENAI_API_KEY=text_of_super_secret_api_key
```

- You won't be able to see the api key again once you close the dialog. If you don't store it and lose it, you'll need to create a new API key.  

Step 1: Create a New Conda Environment

- Open your terminal (or Anaconda Prompt if you are on Windows).
- Create a new Conda environment and activate it:

```
conda create -n haystack_env python=3.9
conda activate haystack_env
```

See: https://haystack.deepset.ai/tutorials/27_first_rag_pipeline

Once you've activated the haystack environment, install Jupyter, Haystack, and other necessary packages in your Conda environment:

```
conda install -c conda-forge jupyter
pip install openai python-dotenv
pip install torch
pip install haystack-ai
pip install "datasets>=2.6.1"
pip install "sentence-transformers>=2.2.0"
```

Open jupyter

In [1]:
# cell 1: let's haystack know you are running a tutorial
from haystack.telemetry import tutorial_running

tutorial_running(27)


In [2]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()


In [3]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]


In [4]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()


C:\Users\Miles\anaconda3\envs\haystack_env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

151

In [6]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")


In [7]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store)


In [8]:
from haystack.components.builders import PromptBuilder

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)


In [9]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

from getpass import getpass
from haystack.components.generators import OpenAIGenerator

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
generator = OpenAIGenerator(model="gpt-3.5-turbo")


In [10]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [11]:
question = "What does Rhodes Statue look like?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Rhodes Statue, also known as the Colossus of Rhodes, was a statue of the Greek sun-god Helios. According to scholars, while the full appearance of the statue is unknown, they do have a good idea of what the head and face looked like. The head would have had curly hair with evenly spaced spikes of bronze or silver flame radiating, similar to the images found on contemporary Rhodian coins. The statue stood approximately 70 cubits, or 33 meters (108 feet) high, making it the tallest statue in the ancient world.


In [12]:
examples = [
    "Where is Gardens of Babylon?",
    "Why did people build Great Pyramid of Giza?",
    "What does Rhodes Statue look like?",
    "Why did people visit the Temple of Artemis?",
    "What is the importance of Colossus of Rhodes?",
    "What happened to the Tomb of Mausolus?",
    "How did Colossus of Rhodes collapse?",
]


In [13]:
for question in examples:
    print(question)
    response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})
    print(response["llm"]["replies"][0])


Where is Gardens of Babylon?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Gardens of Babylon are believed to have been located in the ancient city of Babylon, near present-day Hillah, Babil province, in Iraq.
Why did people build Great Pyramid of Giza?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Great Pyramid of Giza was built as a tomb for the Fourth Dynasty pharaoh Khufu.
What does Rhodes Statue look like?


...

The Colossus of Rhodes was an important statue erected to celebrate the successful defense of Rhodes city against an attack by Demetrius Poliorcetes. It was considered one of the Seven Wonders of the Ancient World and was the tallest statue in the ancient world. The statue symbolized the mastery of the sea by the Rhodians, their victory over their enemies, and their autonomy. It also served as a significant votive offering and a representation of their patron god, Helios. Despite its eventual collapse, the Colossus of Rhodes remained a symbol of power and pride for the people of Rhodes.
What happened to the Tomb of Mausolus?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Tomb of Mausolus was destroyed by successive earthquakes from the 12th to the 15th century. Additionally, parts of the tomb were used for fortifying Bodrum Castle by the Knights of St. John of Rhodes around 1494, and later parts of the remaining tomb were broken up and used in the castle walls when it was strengthened in 1522 to defend against a potential Turkish invasion.
How did Colossus of Rhodes collapse?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Colossus of Rhodes collapsed due to a significant earthquake in 226 BC. The earthquake caused damage to large portions of Rhodes, including the statue itself, which snapped at the knees and fell over onto the land.
